In [1]:
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm

In [6]:
file_path_list = {}
file_path_list['AVE'] = {}
file_path_list['CTA'] = {}
file_path_list['DC'] = {}
file_path_list['DI'] = {}
file_path_list['ER'] = {}
file_path_list['RE'] = {}
file_path_list['SM'] = {}

In [7]:
file_path_list['AVE']['oa_mine'] = '/data/home/wangys/MELD/dataset/AVE/oa_mine_train_small.json'
file_path_list['CTA']['SimTab'] = '/data/home/wangys/MELD/dataset/CTA/SimTab_Train_few.json'
# file_path_list['CTA']['WebTable'] = '/data/home/wangys/MELD/dataset/CTA/webtable_train_few.json' ## selected-file
file_path_list['CTA']['WebTable'] = '/data/home/wangys/DataSelection-IF/train/CTA/WebTable/WebTable-train.json'

file_path_list['DC']['hospital'] = '/data/home/wangys/MELD/dataset/DC/hospital-train-MoE.json'
file_path_list['DC']['rayyan'] = '/data/home/wangys/MELD/dataset/DC/rayyan-train-MoE.json'
file_path_list['DC']['beer'] = '/data/home/wangys/MELD/dataset/DC/beer-train-MoE.json'
# file_path_list['RE']['RE'] = '/data/home/wangys/MELD/dataset/RE/RE-train.json' ## selected-file
file_path_list['RE']['RE'] = '/data/home/wangys/DataSelection-IF/train/RE/RE-train.json'

In [8]:
file_path_list['DI']['amazon'] = '/data/home/wangys/MELD/dataset/DI/amazon_train_output_wide.json'
file_path_list['DI']['walmart'] = '/data/home/wangys/MELD/dataset/DI/walmart_train_output_wide.json'
file_path_list['DI']['restaurant'] = '/data/home/wangys/MELD/dataset/DI/restaurant_train_output_wide.json'

In [9]:
file_path_list['ER']['semi-text-w'] = '/data/home/wangys/MELD/dataset/ER/semi-text-w-train-MoE.json'
file_path_list['ER']['semi-text-c'] = '/data/home/wangys/MELD/dataset/ER/semi-text-c-train-MoE.json'
file_path_list['ER']['amazon-google'] = '/data/home/wangys/MELD/dataset/ER/amazon-google-train.json'
file_path_list['ER']['walmart-amazon'] = '/data/home/wangys/MELD/dataset/ER/walmart_amazon_train_output.json'
file_path_list['ER']['wdc'] = '/data/home/wangys/MELD/dataset/ER/wdc_all_train_output.json'
file_path_list['ER']['abt-buy'] = '/data/home/wangys/MELD/dataset/ER/ant_buy_train_output.json'
file_path_list['SM']['CMS'] = '/data/home/wangys/MELD/dataset/SM/CMS_train_few_output.json'
file_path_list['SM']['synthea'] = '/data/home/wangys/MELD/dataset/SM/synthea_train_few_output.json'

In [10]:
file_path_list_test = {}
file_path_list_test['AVE'] = {}
file_path_list_test['CTA'] = {}
file_path_list_test['DC'] = {}
file_path_list_test['DI'] = {}
file_path_list_test['ER'] = {}
file_path_list_test['RE'] = {}
file_path_list_test['SM'] = {}

In [11]:
file_path_list_test['AVE']['oa_mine'] = '/data/home/wangys/MELD/dataset/AVE/oa_mine_test_small.json'
file_path_list_test['CTA']['SimTab'] = '/data/home/wangys/MELD/dataset/CTA/SimTab_test_few.json'
# file_path_list['CTA']['WebTable'] = '/data/home/wangys/MELD/dataset/CTA/webtable_train_few.json' ## selected-file
file_path_list_test['CTA']['WebTable'] = '/data/home/wangys/MELD/dataset/CTA/WebTable_Test_few.json'

file_path_list_test['DC']['hospital'] = '/data/home/wangys/MELD/dataset/DC/hospital-test.json'
file_path_list_test['DC']['rayyan'] = '/data/home/wangys/MELD/dataset/DC/rayyan-test-20.json'
file_path_list_test['DC']['beer'] = '/data/home/wangys/MELD/dataset/DC/beer_test.json'
# file_path_list['RE']['RE'] = '/data/home/wangys/MELD/dataset/RE/RE-train.json' ## selected-file
file_path_list_test['RE']['RE'] = '/data/home/wangys/DataSelection-IF/train/RE/RE-train.json'

In [12]:
file_path_list_test['DI']['amazon'] = '/data/home/wangys/MELD/dataset/DI/amazon_test_output_wide.json'
file_path_list_test['DI']['walmart'] = '/data/home/wangys/MELD/dataset/DI/walmart_test_output_wide.json'
file_path_list_test['DI']['restaurant'] = '/data/home/wangys/MELD/dataset/DI/restaurant_test_output_wide.json'

In [13]:
file_path_list_test['ER']['semi-text-w'] = '/data/home/wangys/MELD/dataset/ER/semi-text-w-test-MoE.json'
file_path_list_test['ER']['semi-text-c'] = '/data/home/wangys/MELD/dataset/ER/semi-text-c-test-MoE.json'
file_path_list_test['ER']['amazon-google'] = '/data/home/wangys/MELD/dataset/ER/amazon-google-test.json'
file_path_list_test['ER']['walmart-amazon'] = '/data/home/wangys/MELD/dataset/ER/walmart_amazon_test_output.json'
file_path_list_test['ER']['wdc'] = '/data/home/wangys/MELD/dataset/ER/wdc_all_test_output.json'
file_path_list_test['ER']['abt-buy'] = '/data/home/wangys/MELD/dataset/ER/ant_buy_test_output.json'
file_path_list_test['SM']['CMS'] = '/data/home/wangys/MELD/dataset/SM/CMS_test_few_output.json'
file_path_list_test['SM']['synthea'] = '/data/home/wangys/MELD/dataset/SM/synthea_test_few_output.json'

In [ ]:
df_all = pd.DataFrame()
for task in tqdm(file_path_list):
    for dataset in file_path_list[task].keys():
        raw_file = pd.read_json(file_path_list[task][dataset])
        # raw_file
        ppl_array = np.zeros(len(raw_file))
        for process_num in range(1,9,1): ## maximum of k process
            if os.path.exists('../ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num)): ## i-th gradient 
                ppl_df = pd.read_csv('../ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num),index_col=0)
                for index,row in ppl_df.iterrows():
                    ppl_array[index] = row[0]
        if np.sum(ppl_array)>10: ## 存在Ppl结果
            top_k = np.argsort(ppl_array)[::-1][:200]
            df_select = raw_file.iloc[top_k].reset_index(drop=True)
        else:
            print('ppl do not exist for {}/{}'.format(task,dataset))
            df_select = raw_file.sample(n=200,random_state=42).reset_index(drop=True)
        df_select = df_select.iloc[:,:3]
        df_select.columns = ['instruction','input','output']
        df_select['task'] = task
        df_select['dataset'] = dataset
        df_all = pd.concat([df_all,df_select])

In [135]:
## For Test
df_all = pd.DataFrame()
for task in tqdm(file_path_list_test):
    for dataset in file_path_list_test[task].keys():
        raw_file = pd.read_json(file_path_list_test[task][dataset])
        # print(raw_file.columns)
        if 'instruction' not in raw_file.columns:
            raw_file.columns = ['instruction', 'input', 'output'] ## for [0,1,2]
        if 'index' not in raw_file.columns:
            raw_file['index'] = raw_file.index
        # raw_file
            # print('ppl do not exist for {}/{}'.format(task,dataset))
            # df_select = raw_file.sample(n=200,random_state=42).reset_index(drop=True)
            
        df_select = raw_file
        # df_select.columns = ['instruction','input','output']
        df_select['task'] = task
        df_select['dataset'] = dataset
        df_all = pd.concat([df_all,df_select])

  0%|          | 0/7 [00:00<?, ?it/s]

In [136]:
df_all

,instruction,input,output,index,task,dataset
0,You are a world-class expert for extracting in...,,"{""Brand"": ""Shout""}",0,AVE,oa_mine
1,You are a world-class expert for extracting in...,,"{""Specific uses"": ""Stain Remover""}",1,AVE,oa_mine
2,You are a world-class expert for extracting in...,,"{""Item form"": ""Liquid Refill""}",2,AVE,oa_mine
3,You are a world-class expert for extracting in...,,"{""Net content"": ""60 oz""}",3,AVE,oa_mine
4,You are a world-class expert for extracting in...,,"{""Pack size"": ""2 pk""}",4,AVE,oa_mine
...,...,...,...,...,...,...
5923,You are an expert in detecting if two columns ...,,{'Output': 'dismatch'},5923,SM,synthea
5924,You are an expert in detecting if two columns ...,,{'Output': 'dismatch'},5924,SM,synthea
5925,You are an expert in detecting if two columns ...,,{'Output': 'dismatch'},5925,SM,synthea
5926,You are an expert in detecting if two columns ...,,{'Output': 'dismatch'},5926,SM,synthea


In [137]:
df_all.reset_index(drop=True).to_csv('../train/router/test.csv')

In [139]:
def Pseudo_Label_Router(row):
    expert = '{}/{}'.format(row['task'],row['dataset'])
    return [expert]
df_all['labels'] = df_all.apply(Pseudo_Label_Router,axis=1)
df_all['text'] = df_all['instruction']

In [141]:
df_all[['text','labels']].to_csv('../train/router/test_guided.csv')

In [53]:
df_all.reset_index(drop=True).to_csv('../train/router/train.csv')

In [4]:
from transformers import RobertaTokenizer
import pandas as pd
from tqdm.notebook import tqdm
# prompt_opt = pd.read_json('/home/yanmy/LLM_ER/data/dblp-acm/feature_align/dblp-acm-train-dpo.json') ## replace with your training file

tokenizer = RobertaTokenizer.from_pretrained('../../roberta-base/') ## replace with any LM


token_lengths = [len(tokenizer.encode(text, add_special_tokens=True)) for text in tqdm(df_all['output'].to_list())]

print("Token Lengths:", max(token_lengths))

  0%|          | 0/68097 [00:00<?, ?it/s]

Token Lengths: 307


## Expert List for Query

In [5]:
expert_path_list = {}
expert_path_list['AVE'] = {}
expert_path_list['CTA'] = {}
expert_path_list['DC'] = {}
expert_path_list['DI'] = {}
expert_path_list['ER'] = {}
expert_path_list['RE'] = {}
expert_path_list['SM'] = {}

In [6]:
# expert_path_list['AVE']['oa_mine'] = '/data/home/wangys/LLaMA-Factory-main/lora_weight/oa_mine/oa_mine_train_small'
expert_path_list['AVE']['oa_mine'] = '/data/home/wangys/DataSelection-IF/lora/mistral-7B/AVE/oa_mine/select'

expert_path_list['CTA']['SimTab'] = '/data/home/wangys/MELD/lora_weight/expert/CTA_SimTab_train_init'
expert_path_list['CTA']['WebTable'] = '/data/home/wangys/MELD/lora_weight/expert/CTA_WebTable_train_init'

expert_path_list['DC']['hospital'] = '/data/home/wangys/MELD/lora_weight/expert/hospital_train-MoE-Add'
expert_path_list['DC']['rayyan'] = '/data/home/wangys/MELD/lora_weight/expert/rayyan_train-MoE-Add'
expert_path_list['DC']['beer'] = '/data/home/wangys/MELD/lora_weight/expert/beer_train-MoE-Add'
expert_path_list['RE']['RE'] = '/data/home/wangys/MELD/lora_weight/expert/RE-MoE-Add'

In [7]:
expert_path_list['DI']['amazon'] = '/data/home/wangys/MELD/lora_weight/expert/amazon_train-MoE-Add'
expert_path_list['DI']['walmart'] = '/data/home/wangys/MELD/lora_weight/expert/walmart_train-MoE-Add'
expert_path_list['DI']['restaurant'] = '/data/home/wangys/MELD/lora_weight/expert/restaurant_train-MoE-Add'

In [8]:
expert_path_list['ER']['semi-text-w'] = '/data/home/wangys/MELD/lora_weight/expert/semi_text_w-MoE-Add'
expert_path_list['ER']['semi-text-c'] = '/data/home/wangys/MELD/lora_weight/expert/semi_text_c-MoE-Add'
expert_path_list['ER']['amazon-google'] = '/data/home/wangys/MELD/lora_weight/expert/amazon_google-MoE-Add'
expert_path_list['ER']['walmart-amazon'] = '/data/home/wangys/MELD/lora_weight/expert/walmart_amazon-MoE-Add'
expert_path_list['ER']['wdc'] = '/data/home/wangys/MELD/lora_weight/expert/wdc_all-MoE-Add'
expert_path_list['ER']['abt-buy'] = '/data/home/wangys/MELD/lora_weight/expert/ant_buy-MoE-Add'
expert_path_list['SM']['CMS'] = '/data/home/wangys/MELD/lora_weight/expert/CMS_train-MoE-Add'
expert_path_list['SM']['synthea'] = '/data/home/wangys/MELD/lora_weight/expert/synthea_train-MoE-Add'

In [71]:
lora_count = 1
expert_index = {}
for task in expert_path_list:
    for dataset in expert_path_list[task]:
        expert_name = '{}/{}'.format(task,dataset)
        expert_index[expert_name] = {}
        expert_index[expert_name]['index'] = lora_count
        expert_index[expert_name]['lora_path'] = expert_path_list[task][dataset]
        lora_count += 1

In [64]:
expert_index_path = '../train/router/expert_index.npy'
expert_index = np.load(expert_index_path,allow_pickle=True).item()

In [72]:
np.save('../train/router/expert_index.npy',expert_index)

In [115]:
# result = pd.read_csv('../train/router/train_output_guided.csv',index_col=0)
result = pd.read_csv('../train/router/train_output.csv',index_col=0)

In [93]:
row_example = result.iloc[3]

In [104]:
# eval(row_example[10].strip())
for x,y in row_example[5:].items():
    print(x)

AVE/oa_mine
CTA/SimTab
CTA/WebTable
DC/hospital
DC/rayyan
DC/beer
DI/amazon
DI/walmart
DI/restaurant
ER/semi-text-w
ER/semi-text-c
ER/amazon-google
ER/walmart-amazon
ER/wdc
ER/abt-buy
RE/RE
SM/CMS
SM/synthea


In [116]:
def AST(row):
    output = row['output'].strip()
    
    output_item = list(eval(output).values())[0]
    
    positive = []
    # try:
    #     predict = row['predict'].strip()
    #     predict_item = list(eval(predict).values())[0]
    # except:
    #     predict_item = ''
    # row['output_item'] = str(output_item).lower().strip()
    # row['predict_item'] = str(predict_item).lower().strip()
    # return row
    ground_truth = []
    for x,y in row[5:].items():
        try:
            predict = row[x].strip()
            predict_item = list(eval(predict).values())[0]
        except:
            predict_item = ''
        output_item_final = str(output_item).lower().strip()
        predict_item_final = str(predict_item).lower().strip()
        if(output_item_final==predict_item_final):
            ground_truth.append(x)
        # else:
        #     print(output_item_final,predict_item_final,x)
    return ground_truth
# AST(result.iloc[422])
result['labels'] = result.apply(AST,axis=1)

<string>:1: SyntaxWarning: invalid escape sequence '\_'
<string>:1: SyntaxWarning: invalid escape sequence '\_'
<string>:1: SyntaxWarning: invalid escape sequence '\_'
<string>:1: SyntaxWarning: invalid escape sequence '\_'
<string>:1: SyntaxWarning: invalid escape sequence '\_'
<string>:1: SyntaxWarning: invalid escape sequence '\}'
<string>:1: SyntaxWarning: invalid escape sequence '\ '
<string>:1: SyntaxWarning: invalid escape sequence '\_'
<string>:1: SyntaxWarning: invalid escape sequence '\_'
<string>:1: SyntaxWarning: invalid escape sequence '\_'
<string>:1: SyntaxWarning: invalid escape sequence '\_'
<string>:1: SyntaxWarning: invalid escape sequence '\_'
<string>:1: SyntaxWarning: invalid escape sequence '\_'
<string>:1: SyntaxWarning: invalid escape sequence '\_'
<string>:1: SyntaxWarning: invalid escape sequence '\_'
<string>:1: SyntaxWarning: invalid escape sequence '\_'
<string>:1: SyntaxWarning: invalid escape sequence '\_'
<string>:1: SyntaxWarning: invalid escape sequen

In [120]:
result[['instruction','labels']].iloc[663,1]

['DC/hospital', 'DC/rayyan', 'DC/beer']

In [121]:
result_output = result[['instruction','labels']]
result_output.columns = ['text', 'labels']
result_output = result_output.sample(frac=1)

In [122]:
result_output.iloc[:3300].to_csv('../train/router/train_guided.csv')
result_output.iloc[3300:].to_csv('../train/router/valid_guided.csv')

In [144]:
import torch
logits = torch.load('../train/router/logits.pkl')
id2label = torch.load('../train/router/id2label.pkl')

/tmp/ipykernel_2094896/900560314.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  logits = torch.load('../train/router/logits.pkl')
/tmp/ipykernel_2094896/900560314.py:3:

In [146]:
labels = (torch.sigmoid(torch.tensor(logits)) > 0.5).int()

In [170]:
router_dispatch_result_all = np.argsort(-logits)
for index in range(labels.shape[0]):
    router_dispatch_result = router_dispatch_result_all[index]
    router_dispatch_result = np.sort(router_dispatch_result[:2])
    top_k_expert = [id2label[router_dispatch_result[0]],id2label[router_dispatch_result[1]]]
    df_all.iloc[index,6] = str(top_k_expert)

### Merging Models

In [173]:
df_all.reset_index(drop=True).to_csv('../train/router/test.csv')

- CPU OffLoad

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

In [2]:
from transformers import AutoModelForCausalLM
import torch
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
# 加载基础模型
from peft import PeftModel
base_model = AutoModelForCausalLM.from_pretrained("/data/home/wangys/model/Mistral-7B-Instruct-v0.2",device_map='cpu') ## 专家A,domain expert

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
expert_path_list = {}
expert_path_list['AVE'] = {}
expert_path_list['CTA'] = {}
expert_path_list['DC'] = {}
expert_path_list['DI'] = {}
expert_path_list['ER'] = {}
expert_path_list['RE'] = {}
expert_path_list['SM'] = {}
# expert_path_list['AVE']['oa_mine'] = '/data/home/wangys/LLaMA-Factory-main/lora_weight/oa_mine/oa_mine_train_small'
expert_path_list['AVE']['oa_mine'] = '/data/home/wangys/DataSelection-IF/lora/mistral-7B/AVE/oa_mine/select'

expert_path_list['CTA']['SimTab'] = '/data/home/wangys/MELD/lora_weight/expert/CTA_SimTab_train_init'
expert_path_list['CTA']['WebTable'] = '/data/home/wangys/MELD/lora_weight/expert/CTA_WebTable_train_init'

expert_path_list['DC']['hospital'] = '/data/home/wangys/MELD/lora_weight/expert/hospital_train-MoE-Add'
expert_path_list['DC']['rayyan'] = '/data/home/wangys/MELD/lora_weight/expert/rayyan_train-MoE-Add'
expert_path_list['DC']['beer'] = '/data/home/wangys/MELD/lora_weight/expert/beer_train-MoE-Add'
expert_path_list['RE']['RE'] = '/data/home/wangys/MELD/lora_weight/expert/RE-MoE-Add'
expert_path_list['DI']['amazon'] = '/data/home/wangys/MELD/lora_weight/expert/amazon_train-MoE-Add'
expert_path_list['DI']['walmart'] = '/data/home/wangys/MELD/lora_weight/expert/walmart_train-MoE-Add'
expert_path_list['DI']['restaurant'] = '/data/home/wangys/MELD/lora_weight/expert/restaurant_train-MoE-Add'
expert_path_list['ER']['semi-text-w'] = '/data/home/wangys/MELD/lora_weight/expert/semi_text_w-MoE-Add'
expert_path_list['ER']['semi-text-c'] = '/data/home/wangys/MELD/lora_weight/expert/semi_text_c-MoE-Add'
expert_path_list['ER']['amazon-google'] = '/data/home/wangys/MELD/lora_weight/expert/amazon_google-MoE-Add'
expert_path_list['ER']['walmart-amazon'] = '/data/home/wangys/MELD/lora_weight/expert/walmart_amazon-MoE-Add'
expert_path_list['ER']['wdc'] = '/data/home/wangys/MELD/lora_weight/expert/wdc_all-MoE-Add'
expert_path_list['ER']['abt-buy'] = '/data/home/wangys/MELD/lora_weight/expert/ant_buy-MoE-Add'
expert_path_list['SM']['CMS'] = '/data/home/wangys/MELD/lora_weight/expert/CMS_train-MoE-Add'
expert_path_list['SM']['synthea'] = '/data/home/wangys/MELD/lora_weight/expert/synthea_train-MoE-Add'

In [4]:
peft_model_id = "/data/home/wangys/LLaMA-Factory-main/lora_weight/MoE/add/Mistral/amazon_google-MoE-Add"
model = PeftModel.from_pretrained(base_model, model_id=peft_model_id,adapter_name='amazon-google')

In [5]:
for task in expert_path_list:
    for dataset in expert_path_list[task]:
        model.load_adapter(expert_path_list[task][dataset],adapter_name='{}--{}'.format(task,dataset))

In [3]:
df_all = pd.read_csv('../train/router/test.csv',index_col=0)

In [7]:
for expert_combination in tqdm(list(df_all['labels'].unique())):
    expert_0,expert_1 = eval(expert_combination.replace('/','--'))
    # print(expert_0,expert_1)
    model.add_weighted_adapter(
        adapters=[expert_0,expert_1],
        weights=[1,1],
        adapter_name='{}|{}'.format(expert_0,expert_1),
        combination_type="cat"
    )

  0%|          | 0/84 [00:00<?, ?it/s]

In [9]:
adapter_list = []
for expert_combination in tqdm(list(df_all['labels'].unique())):
    expert_0,expert_1 = eval(expert_combination.replace('/','--'))
    adapter_list.append('{}|{}'.format(expert_0,expert_1))

model.save_pretrained(save_directory='/data/home/wangys/Expert_Combination',selected_adapters=adapter_list)

  0%|          | 0/84 [00:00<?, ?it/s]

In [12]:
df_all.iloc[:,:-1].to_csv('../train/router/test.csv')

In [1]:
import shutil
import os

# 假设三个文件分别为file1.txt, file2.txt, file3.txt
files_to_copy = ['/data/home/wangys/LLaMA-Factory-main/lora_weight/MoE_CT/add/Mistral/restaurant-MoE-CT/tokenizer.model', '/data/home/wangys/LLaMA-Factory-main/lora_weight/MoE_CT/add/Mistral/restaurant-MoE-CT/tokenizer_config.json', '/data/home/wangys/LLaMA-Factory-main/lora_weight/MoE_CT/add/Mistral/restaurant-MoE-CT/special_tokens_map.json']
folder_A = '/data/home/wangys/Expert_Combination'

for root, dirs, files in os.walk(folder_A):
    if root != folder_A:  # 排除folder A本身
        for file in files_to_copy:
            shutil.copy(file, root)